In [1]:
__author__='h3imdallr'

In [2]:
"""
@ Obejectives: STL decomposition, visulaization with PLOTLY
@ Requirements:
- python 2.7.x
- statsmodels 0.6.1+ installed
- anom_detect (originally pycularity, but with stasmodels.api, not rpy2)
"""

'\n@ Obejectives: STL decomposition, visulaization with PLOTLY\n@ Requirements:\n- python 2.7.x\n- statsmodels 0.6.1+ installed\n- anom_detect (originally pycularity, but with stasmodels.api, not rpy2)\n'

plotly is an online analytics and data visualization tool. They provide beautiful&interactive visualization functions.
Here in this notebook, using graph objects of plotly is desscribe, including subplotting method.

In [3]:
import datetime
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm  
from statsmodels.tsa.seasonal import seasonal_decompose

import plotly
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot

** Data Loading: **

In [4]:
# data source: Numenta Anomaly Benchmark(NAB) 
# https://github.com/numenta/NAB/tree/master/data
n_file = 'machine_temperature_system_failure'
timeS_DF = pd.read_csv('./data/%s.csv' % n_file, usecols=['timestamp', 'value'])

print ">>>data loading successful. \n dataframe collected: \n", timeS_DF.head()

>>>data loading successful. 
 dataframe collected: 
             timestamp      value
0  2013-12-02 21:15:00  73.967322
1  2013-12-02 21:20:00  74.935882
2  2013-12-02 21:25:00  76.124162
3  2013-12-02 21:30:00  78.140707
4  2013-12-02 21:35:00  79.329836


In [5]:
"""  STL Decomposition  """
timeS_DF.columns = ["timestamp", "value"]
timeS_DF['timestamp'] = pd.to_datetime(timeS_DF['timestamp'])
timeS_DF.set_index('timestamp', inplace=True)

# >>> Maybe Resammple HERE <<<
flow = timeS_DF['value']
flow.head(10)

timestamp
2013-12-02 21:15:00    73.967322
2013-12-02 21:20:00    74.935882
2013-12-02 21:25:00    76.124162
2013-12-02 21:30:00    78.140707
2013-12-02 21:35:00    79.329836
2013-12-02 21:40:00    78.710418
2013-12-02 21:45:00    80.269784
2013-12-02 21:50:00    80.272828
2013-12-02 21:55:00    80.353425
2013-12-02 22:00:00    79.486523
Name: value, dtype: float64

In [6]:
flow.dropna(inplace=True)

**Applying STL decompose:**

In [7]:
res = seasonal_decompose(flow, freq = 24*60/3)

trend = res.trend
seasonal = res.seasonal
residual = res.resid

STL_DF = pd.concat([residual, seasonal, trend], axis=1, keys=['residual', 'seasonal', 'trend'])
print """>>> STL Decomposition Compeleted """

>>> STL Decomposition Compeleted 


**STL decompositoin graph using PLOTLY**

To use plotly offline in jupyter, you need to initiate  Plotly Notebook mode  as below.  [link](https://plot.ly/python/offline/)  

In [8]:
""" Initialize Plotly Notebook mode """
init_notebook_mode(connected=True)

In [9]:
if True: 
    """optional for selecting time"""
    output_DF = STL_DF[
        (STL_DF.index >= datetime.datetime(2013, 12,8))
        & (STL_DF.index <= datetime.datetime(2014, 1,1)) ]
else: 
    output_DF = STL_DF.copy()

""" Create Figure """
x = output_DF.index

# Initialize trace of streaming plot by embedding the unique stream_id
trace1 = go.Scatter(
    x=x,
    y=output_DF['trend'],
    mode='lines',
    line=dict(color='steelblue'),
    name='TREND'
)
trace2 = go.Scatter(
    x=x,
    y=output_DF['seasonal'],
    mode='lines',
    # marker=dict(size=15, color='salmon', symbol = 'star' ),
    line=dict(color='orange'),
    name='SEASONALITY'
)
trace3 = go.Scatter(
    x=x,
    y=output_DF['residual'],
    mode='lines',
    line=dict(color='salmon'),
    name='RESIDUAL'
)

In [10]:
""" SUBPLOTTING """
fig = tls.make_subplots(rows=3, cols=1, subplot_titles=['TREND', 'Seasonality', 'Residual']
                        # shared_xaxes=False,
                        # print_grid=True
                        # shared_yaxes=True
                        )
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 2, 1)
fig.append_trace(trace3, 3, 1)

fig['layout'].update( title='STL deocomposition (Static Ver.)',
                    # height=400, width=400,
                     showlegend=False,
                     # legend=dict(x=0.02, y=1)
                     )

# Send fig to Plotly, initialize streaming plot, open new tab
"""Online"""
# url = py.plot(fig, filename='STL-Decomposition-Static', auto_open=False)
# embed_code = tls.get_embed(url)
# print ">>> embed_code: ", embed_code

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]



'Online'

** Plot in jupyter notebook **

In [11]:
"""Offline"""
iplot(fig, filename='outputs/STLdecompose_%s.html' % n_file)

Thanks to its inteactive functions, you can **view values/expand/pan/save** the figure. Try mouseover on the figure

** Analysis: (same as STL_decomposition_NABdata.ipynb in directory)**


We can extract trend/seasonality/residual from original time-series, `flow`.  

The simple understanding between stl components and original signal:

![stl](stl_example.png)
[source](https://anomaly.io/seasonal-trend-decomposition-in-r/)

The good use of each:
* trend - help indentifiy macro move
* seasonality - help indentify repeating time-series pattern
* residual - help detect randomness, abnormal activities, or level of stationary of the signal.

This STL decomposition analysis provides basis for **time-series anomaly detection**, which will be introduced.

(End of Document)
